In [1]:
import pandas as pd
import os
import re
from ipysheet import from_dataframe, to_dataframe


In [14]:
# NFCU
#path = "./in/transactionsNFCUcc.CSV"
#df = pd.read_csv(path)
#df["Date"] = pd.to_datetime(df["Transaction Date"])
#df.rename(columns={"Description": "Payee"}, inplace=True)
#df.to_csv("./out/nfcuvisa.csv")

path = "./in/transactionsNFCUchk.CSV"
df = pd.read_csv(path)
df["Date"] = pd.to_datetime(df.Date)
df.drop("No.", axis=1, inplace=True)
df.rename(columns={"Description": "Payee"}, inplace=True)
df.to_csv("./out/nfcudebit.csv")


In [3]:
# Ally
path = "./in/ally.csv"
df = pd.read_csv(path)

df.drop(columns=[" Time", " Type"], inplace=True)
df.rename(columns={" Description": "Payee"}, inplace=True)

df.to_csv("./out/ally.csv")


In [4]:
# Chase
path = "./in/"
for filename in os.listdir(path):
    if re.match("Chase.*.CSV", filename):
        path = path + filename
df = pd.read_csv(path)
df.drop(columns=["Transaction Date", "Category", "Type"], inplace=True)
df.rename(columns={"Post Date": "Date"}, inplace=True)

df.to_csv("./out/Chase.csv")


In [6]:
# Amex
path = "./in/activity.csv"
df = pd.read_csv(path)

df.rename(columns={"Description": "Payee"}, inplace=True)

df.to_csv("./out/amex.csv")


In [8]:
def paypal(paypal_df, all=False):
    paypal_df.rename(
        columns={"date": "Date", "amount": "Amount", "description": "Payee"},
        inplace=True,
    )
    paypal_df["Date"] = pd.to_datetime(paypal_df.Date)
    paypal_df.drop(columns=["id"], inplace=True)
    paypal_df["Amount"] = (
        paypal_df["Amount"]
        .replace("[\$,)]", "", regex=True)
        .replace("[(]", "-", regex=True)
        .astype(float)
    )
    if all:
        paypal_df.to_csv("./out/all_paypal.csv")
    else:
        paypal_df.to_csv("./out/mc_paypal.csv")


def full_view(df):
    # Creates csv for each instututionAccount from fullview transaction aggregation output.
    # fixes dates
    df["Date"] = pd.to_datetime(df["Date"])

    # Creates institution-account column and list of uniques.
    df["Acct"] = df["Institution"] + " " + df["Account"]
    accts = df["Acct"].unique().tolist()

    # Creates each list of transactions per account, only including posted transactions.
    for acct in accts:
        acct_df = df.loc[
            (df["Acct"] == acct)
            & (df["Is Hidden"] == "No")
            & (df["Is Pending"] == "No")
        ]

        # Format for actual by dropping unused columns, renaming payee, and formatting amounts.
        acct_df = acct_df.drop(
            columns=[
                "Acct",
                "Institution",
                "Account",
                "Is Hidden",
                "Category",
                "Is Pending",
            ]
        )
        acct_df = acct_df.rename(columns={"Description": "Payee"})
        acct_df["Amount"] = (
            acct_df.loc[:, ["Amount"]]
            .replace("[\$,)]", "", regex=True)
            .replace("[(]", "-", regex=True)
            .astype(float)
        )

        acct_df.to_csv(f"./out/{acct.replace('*','')}.csv")


path = "in"
if not os.path.exists("out"):
    os.mkdir("out")

for filename in os.listdir(path):
    if "DS_Store" in filename:
        continue

    file_path = os.path.join(path, filename)
    df = pd.read_csv(file_path)

    if re.match("transactions_.*.csv", filename):
        full_view(df)
    elif "recent-transactions" in filename:
        paypal(df, all=True)
    else:
        paypal(df)